In [12]:

import pandas as pd
import json
import pathlib
import sys
import concurrent.futures
import os
import numpy as np


In [13]:

sys.path.insert(0, "..\\controller")
from tile_configuration import TileConfiguration


In [14]:

prefix = pathlib.Path(
    "C:\\Users\\davidek\\microscope_ui\\controller\\photo\\1732508547.7836869"
)

tc = TileConfiguration()
tc.load(prefix / "TileConfiguration.registered.txt")
tc.move_to_origin()
tc.images = tc.images[:10]



In [15]:

def load(filename):
    with open(filename) as r:
        return json.load(r)


In [16]:

d = []

with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    future_to_fname = {
        executor.submit(
            load,
            pathlib.Path("..\\out")
            / image.filename.with_suffix(image.filename.suffix + ".json"),
        ): image.filename
        for image in tc.images
    }

    for future in concurrent.futures.as_completed(future_to_fname):
        filename = future_to_fname[future]
        results = future.result()
        for result in results:
            d.append( (filename, pathlib.Path(result), results[result][0], results[result][1]))


In [17]:
p = pd.DataFrame(d, columns=['fname1', 'fname2', 'tx', 'ty'])


In [18]:
p['mag'] = np.sqrt(p['tx'] ** 2 + p['ty'] ** 2)


In [19]:
p.sort_values(by=['mag'], inplace=True)
#p.reset_index()
#p['index_col'] = p.index



In [20]:
p.to_csv("test.csv")

In [ ]:
def combine_two_images(i1, i2, tx, ty):
    # find the boundary of the union
    # paste the two images into the union
